In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime
import seaborn as sns
import re

pd.set_option('display.max_rows', 135)
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', '{:.2f}'.format)

import psycopg2

In [2]:
conn = psycopg2.connect("host=172.32.1.126 port=5432 user=linkshops password=$dpdlvmflf4 dbname=LINKSHOPS-20211121")

curs = conn.cursor()

In [3]:
from_date = '2021-05-01'
to_date = '2021-08-01'

# 0. DataLoad

## 1) order_products

In [4]:
SQL= "select \"id\"," + (
    "\"buyerId\",") + (
    "\"brand\",") + ( 
    "\"orderId\",") + (
    "\"KRW\",") + (
    "\"finalKRW\",") + ( 
    "\"totalKRW\",") + (
    "\"finalTotalKRW\",") + ( 
    "\"finalQuantity\",") + ( 
    "\"quantity\",") + (
    "\"cmUnclePickupQuantity\",") + (
    "\"cmInspectionQuantity\",") + (
    "\"cmInputQuantity\",") + (
    "\"cmOutputQuantity\",") + (
    "\"cmDefectQuantity\",") + (
    "\"cmRegularQuantity\",") + (
    "\"createdAt\"") + (
    " from order_products WHERE ") + (
    f"\"createdAt\" > '{from_date}' AND ") + (
    f"\"createdAt\" < '{to_date}'")

curs.execute(SQL)
dataframe_order_prod = DataFrame(curs)

dataframe_order_prod.columns = ['orderProductId',
                                'buyerId',
                                'brand',
                                'orderId',
                                'KRW',
                                'finalKRW',
                                'cusumOrderAmount',
                                'cusumPurchaseAmount',
                                'cusumPurchaseQuantity',
                                'cusumOrderQuantity',
                                'unclepickup',
                                'inspection',
                                'input',
                                'output',
                                'defect',
                                'regular',
                                'createdAt']

## 2) tb_order_fee

In [5]:
SQL = "select * from tb_order_fee WHERE " + (
    f"\"created_at\" > '{from_date}' AND ") + (
    f"\"created_at\" < '{to_date}'")

curs.execute(SQL)

dataframe_fee = DataFrame(curs)

SQL = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'tb_order_fee'"

curs.execute(SQL)

dataframe_fee.columns = [i[0] for i in curs]

dataframe_fee.iloc[:,9:] = dataframe_fee.iloc[:,9:].astype('float')

## 3) order_product_fee

In [6]:
SQL = "select *" + (
    " from tb_order_product_fee WHERE ") + (
    f"\"created_at\" > '{from_date}' AND ") + (
    f"\"created_at\" < '{to_date}'")

curs.execute(SQL)
dataframe_prod_fee = DataFrame(curs)

In [7]:
SQL = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'tb_order_product_fee'"

curs.execute(SQL)

dataframe_prod_fee.columns = [i[0] for i in curs]

## 4) orders

In [8]:
SQL = "select * " + ( 
    " from orders WHERE ") + (
    f"\"createdAt\" > '{from_date}' AND ") + (
    f"\"createdAt\" < '{to_date}' AND ") + (
    "\"status\" != -1")

curs.execute(SQL)

dataframe_order = pd.DataFrame(curs)

In [9]:
SQL = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name = 'orders'"

curs.execute(SQL)

dataframe_order.columns = [i[0] for i in curs]

In [10]:
dtypes = dataframe_order.apply(lambda x : x.dtype)

In [11]:
for i in dtypes[dtypes == 'object'].index:
    try:
        dataframe_order.loc[:, i] = dataframe_order.loc[:, i].astype('float')
    except :
        pass

# 1. EDA

## 1) 교차 분석 

앞전에 실시한 orders EDA에서, init 및 final이 도출되는 산식은 이미 도출한 바 있다.

이를 orders가 아닌, 주문번호 단위까지 확장한 order_product로 일반화하기 위해 현재 EDA를 실시한다.

다만, 4대 요소 중 '배송비'는 주문번호 단위가 아닌 주문단위로 묶어서 발송되는 특성이 있어 계산이 안된다.

**-> 수정 : tb_product_shipment_logs를 참고하면 상품주문단위의 배송비를 연계 가능하나, 테스트서버엔 해당 데이터를 확보하기 어렵다.**

따라서 사입수수료와 상품가액, 검수검품비, 세금만 우선적으로 계산한다.

### (1) orders, tb_order_fee와 교차분석

In [12]:
cols = list(set(dataframe_prod_fee.columns).difference(
    set(['id','created_at','creator_id','updated_at',' updater_id', 'use_yn','order_id', 'item_type','item_id'])))

cols.sort()

In [13]:
dataframe_prod_fee[cols] = dataframe_prod_fee[cols].astype('float')

dataframe_prod_fee = dataframe_prod_fee.rename({'item_id' : 'orderProductId'}, axis = 1)

In [14]:
order_prod_fee = pd.merge(dataframe_order_prod, 
                        dataframe_prod_fee,
                         on = ['orderProductId'])

#### - 가설 : 아래의 관계는 성립할것이다.

1) orders의 finalHandlingFee = tb_order_prod_fee의 unit_handling_Fee * order_product의 finalQuantity

2) orders의 finalSubtotalKRW = tb_order_prod_fee의 unit_product_supply * order_product의 finalQuantity

3) orders의 finalTax = tb_order_prod_fee의 unit_product_vat * order_product의 finalQuantity

4) order_fee의 quality_inspection_fee_amount = tb_order_prod_fee의 unit_quality_inspection_fee_amount * order_product의 finalQuantity

In [15]:
dataframe_order_prod[['unclepickup', 'inspection', 'input', 'output', 'defect', 'regular']].apply(lambda x : x.isna().value_counts())

unclepickup  inspection   input  output  defect  regular
True        584826      584826  584826  584826  585003   584826
False       149882      149882  149882  149882  149705   149882

In [16]:
order_prod_fee['finalKRW'] = order_prod_fee['finalKRW'].astype('float')
order_prod_fee['cusumPurchaseAmount'] = order_prod_fee['cusumPurchaseAmount'].astype('float')

In [17]:
#finalQualityInspectionFee(최종검수검품비) 계산

order_prod_fee['quality_inspection_fee_amount'] = order_prod_fee['inspection'] * (
                                                  order_prod_fee['unit_quality_inspection_fee_amount'])

In [18]:
#finalHandlingFeeKRW(최종사입료) 계산

order_prod_fee['finalHandlingFeeKRW'] = order_prod_fee['cusumPurchaseQuantity'] * (
                                                    order_prod_fee['unit_handling_fee_amount'])

In [19]:
#finalSubtotalKRW(최종상품액) 계산

order_prod_fee['finalSubtotalKRW'] = order_prod_fee['cusumPurchaseQuantity'] * (
                                                    order_prod_fee['unit_product_supply'])

In [20]:
#finalTaxKRW(최종VAT) 계산

order_prod_fee['finalTaxKRW'] = order_prod_fee['cusumPurchaseQuantity'] * (
                                                    order_prod_fee['unit_product_vat'])

In [21]:
# order_prod_fee를 order_id별로 묶어 더한 후, 이를 orders의 데이터들과 비교한다.

test = pd.merge(dataframe_order.rename({'id' : 'order_id'}, axis = 1),
                        order_prod_fee.groupby('order_id')[cols + ['finalHandlingFeeKRW', 
                                                      'finalSubtotalKRW',
                                                     'finalTaxKRW']].sum().reset_index(),
                        on = 'order_id',
                        suffixes=['', '_unit'])

In [22]:
test = test[np.sort(test.columns)]

In [23]:
test['finalHandlingFeeKRW'] = test['finalHandlingFeeKRW'].astype('float')
test['finalSubtotalKRW'] = test['finalSubtotalKRW'].astype('float')
test['finalTaxKRW'] = test['finalTaxKRW'].astype('float')

In [24]:
# orders의 finalSubtotalKRW와, 파생값으로 신규 생성한 finalSubtotalKRW_unit간 차액 도출

(test['finalSubtotalKRW'] - test['finalSubtotalKRW_unit']).unique()

array([ 0.00000e+00,          nan, -8.50000e+04, -5.00000e+04,
       -4.75000e+05, -4.00000e+04, -1.26003e+05, -2.40000e+04,
       -2.41000e+05, -4.00000e+03, -1.66500e+05, -7.00000e+03,
       -1.30000e+04, -7.50000e+04, -2.44000e+05, -9.35000e+04,
       -4.60000e+04, -1.79000e+05, -1.15000e+05, -9.00000e+03,
       -1.08000e+05, -2.36000e+05, -8.10000e+04, -1.60000e+01,
       -2.50000e+05, -1.00000e+03,  6.40000e+04,  6.50000e+04,
       -7.00000e+02, -2.20000e+05, -1.00000e+05, -2.50000e+04,
       -5.25000e+04, -9.80000e+04, -2.74000e+05, -3.34500e+05,
       -1.00000e+04, -1.80000e+04, -1.60000e+05, -6.50000e+04,
       -2.00000e+03, -2.00000e+04, -1.52500e+05, -1.50000e+05,
       -3.00000e+04, -3.60000e+05, -2.80000e+05, -1.30000e+05,
       -3.40000e+04, -9.50000e+04, -2.70000e+04, -2.12000e+05,
       -3.00000e+05, -2.70000e+05, -1.67500e+05, -4.74500e+05,
       -2.41500e+05, -1.95000e+05])

In [25]:
# 차액이 도출된 사례만 뽑는다

test.loc[((test['finalSubtotalKRW'] - test['finalSubtotalKRW_unit']) != 0) & (
            ~(test['finalSubtotalKRW'] - test['finalSubtotalKRW_unit']).isna()), ['buyerId',
                                                                                    'createdAt',
                                                                                    'order_id',
                                                                                    'finalSubtotalKRW',
                                                                                    'finalSubtotalKRW_unit']]

buyerId                        createdAt  order_id  finalSubtotalKRW  \
291      42191 2021-06-08 08:09:00.280784+00:00   1037164        7024000.00   
1735     42191 2021-07-05 07:53:56.703054+00:00   1059860        1640500.00   
2143     42191 2021-06-07 07:49:38.758389+00:00   1035965        9297000.00   
2267    130052 2021-05-12 09:34:46.140797+00:00   1015644        2843000.00   
2669     16841 2021-05-27 13:59:45.700887+00:00   1027962         113000.00   
2845     95175 2021-07-15 01:21:14.106705+00:00   1067548         248000.00   
3005     42191 2021-05-11 06:46:22.777524+00:00   1014602        9938000.00   
3262    136026 2021-05-07 06:14:34.702034+00:00   1011898          44000.00   
3549     42191 2021-05-05 08:25:17.689831+00:00   1010378        6283500.00   
4541     42191 2021-07-16 06:03:46.772138+00:00   1068389        1138500.00   
4587    126622 2021-05-10 07:43:40.796661+00:00   1013692         309000.00   
4661     42191 2021-06-22 06:26:16.459217+00:00   1049776        8871000.00   
5293     42191 2021-05-26 07:44:06.739557+00:00   1026785       10819000.00   
5322     42191 2021-06-04 07:43:36.627030+00:00   1033979       10374500.00   
6213     42191 2021-05-12 06:54:55.928891+00:00   1015527        9233300.00   
6217     42191 2021-05-11 06:36:30.230738+00:00   1014595        3501000.00   
7527    130052 2021-05-20 08:51:13.578400+00:00   1022226              0.00   
7571     74474 2021-06-24 04:17:13.497002+00:00   1051379         116000.00   
8512     42191 2021-05-24 02:52:18.176748+00:00   1024444        6991000.00   
8630     42191 2021-05-28 08:10:11.530420+00:00   1028319        8044000.00   
8924    130052 2021-05-20 02:07:30.470955+00:00   1021945         200000.00   
8939     42191 2021-05-13 04:11:32.239823+00:00   1016323        9830000.00   
9271    135967 2021-05-13 07:34:47.944059+00:00   1016479         440000.00   
11404   134002 2021-05-26 04:25:10.191927+00:00   1026640         336000.00   
11906    42191 2021-05-25 07:04:45.607978+00:00   1025831        9919000.00   
11951   125872 2021-07-22 05:00:49.441104+00:00   1073107           8000.00   
11988    58242 2021-06-07 03:36:48.338457+00:00   1043624          64000.00   
12003   131506 2021-06-06 21:40:13.827874+00:00   1043721          65000.00   
12986    42191 2021-06-15 07:48:29.083524+00:00   1043898         916800.00   
13007    42191 2021-06-16 07:55:45.265277+00:00   1044829       10999500.00   
13843    42191 2021-07-19 08:22:58.464066+00:00   1070171        2845000.00   
13941    42191 2021-05-04 08:08:12.839018+00:00   1009887       11325000.00   
14082    42191 2021-07-02 07:29:18.515628+00:00   1057932        2899000.00   
14175    42191 2021-07-06 09:15:57.703244+00:00   1061161         950000.00   
14202    42191 2021-06-16 07:59:58.501840+00:00   1044838       13684500.00   
14362    42191 2021-05-04 08:04:16.098340+00:00   1009884        8935100.00   
14541    42191 2021-06-17 08:11:15.184615+00:00   1045737        6286500.00   
15377    42191 2021-05-10 07:04:03.680237+00:00   1013641       10256500.00   
15409   135210 2021-07-27 00:44:38.036833+00:00   1076537         296000.00   
16976    42191 2021-05-06 08:24:15.892063+00:00   1011203        5060000.00   
16977    42191 2021-05-03 03:36:24.092202+00:00   1008962        4845000.00   
16989    42191 2021-07-23 08:27:32.016342+00:00   1074018         404000.00   
16993    42191 2021-07-22 07:35:16.838061+00:00   1073267          77000.00   
17003    42191 2021-07-29 08:11:32.663155+00:00   1078606        6582000.00   
17006    42191 2021-06-29 08:42:49.242641+00:00   1055150       10845000.00   
17007    42191 2021-06-10 08:31:24.661485+00:00   1039371        7590500.00   
17050    42191 2021-06-03 02:41:31.549078+00:00   1033050       10258500.00   
17062    42191 2021-07-12 07:31:18.063119+00:00   1065396         731000.00   
17063    42191 2021-06-11 07:25:34.559114+00:00   1040675        8807500.00   
17070    42191 2021-07-13 07:17:33.688770+00:00   1066249

In [26]:
# orders의 finalTaxKRW, 파생값으로 신규 생성한 finalTaxKRW_unit간 차액 도출

(test['finalTaxKRW'] - test['finalTaxKRW_unit']).unique()

array([ 0.00e+00,       nan, -4.00e+03, -1.26e+04, -2.40e+03, -4.00e+02,
       -1.15e+04, -1.08e+04, -8.10e+03, -2.00e+00,  6.40e+03,  6.50e+03,
       -1.80e+03])

In [27]:
# 차액이 도출된 사례만 뽑는다.

test.loc[((test['finalTaxKRW'] - test['finalTaxKRW_unit']) != 0) & (
            ~(test['finalTaxKRW'] - test['finalTaxKRW_unit']).isna()), ['buyerId',
                                                                    'createdAt',
                                                                    'order_id',
                                                                    'finalTaxKRW',
                                                                    'finalTaxKRW_unit']]

buyerId                        createdAt  order_id  finalTaxKRW  \
2267    130052 2021-05-12 09:34:46.140797+00:00   1015644    284300.00   
2669     16841 2021-05-27 13:59:45.700887+00:00   1027962     11300.00   
2845     95175 2021-07-15 01:21:14.106705+00:00   1067548     24800.00   
3262    136026 2021-05-07 06:14:34.702034+00:00   1011898      4400.00   
7527    130052 2021-05-20 08:51:13.578400+00:00   1022226         0.00   
7571     74474 2021-06-24 04:17:13.497002+00:00   1051379     11600.00   
8924    130052 2021-05-20 02:07:30.470955+00:00   1021945     20000.00   
9271    135967 2021-05-13 07:34:47.944059+00:00   1016479     44000.00   
11404   134002 2021-05-26 04:25:10.191927+00:00   1026640     33600.00   
11988    58242 2021-06-07 03:36:48.338457+00:00   1043624      6400.00   
12003   131506 2021-06-06 21:40:13.827874+00:00   1043721      6500.00   
15409   135210 2021-07-27 00:44:38.036833+00:00   1076537     29600.00   

       finalTaxKRW_unit  
2267          288300.00  
2669           23900.00  
2845           27200.00  
3262            4800.00  
7527           11500.00  
7571           12000.00  
8924           30800.00  
9271           52100.00  
11404          33602.00  
11988              0.00  
12003              0.00  
15409          31400.00

위에서 도출한 subtotal 및 tax 불일치 사례를 L3 Admin을 통해 확인해본 결과 다음의 결론을 얻을 수 있었다.

- CO팀에 의해, 제품의 단위 가격이 변동되는 경우가 자주 발생한다(ex. 9500원 -> 9000원)
- unit_product_supply는 이를 반영하지 않는다.
- tb_order_prod_fee에 'unit_final_product'라는 값이 존재하는데, 이를 반영하면 오차가 줄어드는지 확인이 필요하다.

In [28]:
# unit_final_product_supply는 값 변동이 없을경우 결측값이다.
# 이를 반영하여, unit_final_product_supply가 결측이 아닌 경우엔 해당 값으로 unit_product_supply를 대체한다.

order_prod_fee.loc[~order_prod_fee['unit_final_product_supply'].isna(), 'unit_product_supply'] = (
order_prod_fee.loc[~order_prod_fee['unit_final_product_supply'].isna(), 'unit_final_product_supply'])

order_prod_fee.loc[~order_prod_fee['unit_final_product_supply'].isna(), 'unit_product_vat'] = (
order_prod_fee.loc[~order_prod_fee['unit_final_product_supply'].isna(), 'unit_final_product_vat'])

In [29]:
order_prod_fee['finalSubtotalKRW'] = order_prod_fee['cusumPurchaseQuantity'] * (
                                                    order_prod_fee['unit_product_supply'])

In [30]:
order_prod_fee['finalTaxKRW'] = order_prod_fee['cusumPurchaseQuantity'] * (
                                                    order_prod_fee['unit_product_vat'])

In [31]:
test = pd.merge(dataframe_order.rename({'id' : 'order_id'}, axis = 1),
                        order_prod_fee.groupby('order_id')[cols + ['finalHandlingFeeKRW', 
                                                      'finalSubtotalKRW',
                                                     'finalTaxKRW']].sum().reset_index(),
                        on = 'order_id',
                        suffixes=['', '_unit'])

In [32]:
test = test[np.sort(test.columns)]

In [33]:
test['finalHandlingFeeKRW'] = test['finalHandlingFeeKRW'].astype('float')
test['finalSubtotalKRW'] = test['finalSubtotalKRW'].astype('float')
test['finalTaxKRW'] = test['finalTaxKRW'].astype('float')

In [34]:
(test['finalSubtotalKRW'] - test['finalSubtotalKRW_unit']).unique()

array([      0.,      nan,  -40000., -115000., -108000.,   64000.,
         65000.,  -18000.])

In [35]:
test.loc[((test['finalSubtotalKRW'] - test['finalSubtotalKRW_unit']) != 0) & (
            ~(test['finalSubtotalKRW'] - test['finalSubtotalKRW_unit']).isna()),  ['buyerId',
                                                                                    'createdAt',
                                                                                    'order_id',
                                                                                    'finalSubtotalKRW',
                                                                                    'finalSubtotalKRW_unit']]

buyerId                        createdAt  order_id  finalSubtotalKRW  \
2267    130052 2021-05-12 09:34:46.140797+00:00   1015644        2843000.00   
7527    130052 2021-05-20 08:51:13.578400+00:00   1022226              0.00   
8924    130052 2021-05-20 02:07:30.470955+00:00   1021945         200000.00   
11988    58242 2021-06-07 03:36:48.338457+00:00   1043624          64000.00   
12003   131506 2021-06-06 21:40:13.827874+00:00   1043721          65000.00   
15409   135210 2021-07-27 00:44:38.036833+00:00   1076537         296000.00   

       finalSubtotalKRW_unit  
2267              2883000.00  
7527               115000.00  
8924               308000.00  
11988                   0.00  
12003                   0.00  
15409              314000.00

In [36]:
(test['finalTaxKRW'] - test['finalTaxKRW_unit']).unique()

array([     0.,     nan,  -4000., -11500., -10800.,   6400.,   6500.,
        -1800.])

In [37]:
test.loc[((test['finalTaxKRW'] - test['finalTaxKRW_unit']) != 0) & (
        ~(test['finalTaxKRW'] - test['finalTaxKRW_unit']).isna()), ['buyerId',
                                                                    'createdAt',
                                                                    'order_id',
                                                                    'finalTaxKRW',
                                                                    'finalTaxKRW_unit']]

buyerId                        createdAt  order_id  finalTaxKRW  \
2267    130052 2021-05-12 09:34:46.140797+00:00   1015644    284300.00   
7527    130052 2021-05-20 08:51:13.578400+00:00   1022226         0.00   
8924    130052 2021-05-20 02:07:30.470955+00:00   1021945     20000.00   
11988    58242 2021-06-07 03:36:48.338457+00:00   1043624      6400.00   
12003   131506 2021-06-06 21:40:13.827874+00:00   1043721      6500.00   
15409   135210 2021-07-27 00:44:38.036833+00:00   1076537     29600.00   

       finalTaxKRW_unit  
2267          288300.00  
7527           11500.00  
8924           30800.00  
11988              0.00  
12003              0.00  
15409          31400.00

- 오차 사례가 확연하게 줄어들었다.

- L3 ADMIN 기준으로 확인 결과, 참고값이 0인 경우 파생값이 맞았고, 0이 아닌데 파생값이 0인 경우도 파생값이 맞았다. 
- 다만, 15409 사례는 참고값쪽이 맞아 확인이 필요하다.

In [38]:
(test['finalHandlingFeeKRW'] - test['finalHandlingFeeKRW_unit']).unique()

array([     0.,  -4410.,     nan,  -1090.,  -1920.,   -770.,  -6240.,
        -3500.,  -6490.,  -1500.,  -3600.,  -7740., -13130.,  -5570.,
        -4470.,  -1050., -16180.,  -1350.,  -1820.,  -2610.,  -3340.,
         -980.,  -5960.,  -1900.,  -3050.,  -4610.,   -840.,   -830.,
        -5810.,  -4800.,   -440.,  -3880.,   -870.,  -5860.,   -550.,
        -2100.,  -1370.,  -3540.,  -4930.,  -2110.,   -580.,  -2690.,
        -1850.,  -1700.,  -5490.,  -9930.,  -2980.,  -1320.,   -690.,
        -6080.,  -2500.,  -2830.,  -2470.,   -540.,  -1200.,  -2840.,
        -1130.,  -2750.,  -1280.,  -2790.,  -3660.,  -3220.,  -4750.,
        -4180.,  -4570.,   -250.,  -6570.,  -7730.,  -4130.,  -7500.,
        -1430.,  -2820.,  -3230.,  -1770.,  -3060.,  -5180.,  -2850.,
        -1660.,  -5840.,  -8870.,  -3560.,  -2120.,  -4000.,  -5220.,
        -4150.,  -3740.,  -2040.,  -3200.,  -4550.,  -3960.,  -8360.,
        -4210.,  -9130.,  -3590.,  -4360., -14290.,  -6100.,  -5320.,
        -4840., -108

In [39]:
test.loc[((test['finalHandlingFeeKRW'] - test['finalHandlingFeeKRW_unit']) != 0) & (
    ~(test['finalHandlingFeeKRW'] - test['finalHandlingFeeKRW_unit']).isna()), ['buyerId',
                                                                                'createdAt',
                                                                                'order_id',
                                                                                'finalHandlingFeeKRW',
                                                                                'finalHandlingFeeKRW_unit']]

buyerId                        createdAt  order_id  \
19      133394 2021-05-12 12:16:45.350440+00:00   1015766   
935     118481 2021-06-21 01:00:40.007368+00:00   1047971   
959      70746 2021-06-21 01:12:16.644685+00:00   1048055   
970       1234 2021-06-21 01:20:16.644268+00:00   1048097   
974      34820 2021-06-21 01:20:26.591458+00:00   1048098   
1031     42447 2021-06-21 02:56:23.885576+00:00   1048360   
2054    121664 2021-06-21 02:17:52.870113+00:00   1048251   
2267    130052 2021-05-12 09:34:46.140797+00:00   1015644   
2450     67317 2021-05-12 13:52:34.240159+00:00   1015953   
2963     89631 2021-06-21 01:27:55.058885+00:00   1048137   
3007    131208 2021-05-12 10:44:39.231022+00:00   1015687   
3021    132113 2021-06-21 02:51:58.665154+00:00   1048348   
3356     97151 2021-05-12 14:36:00.702992+00:00   1016042   
3464     77910 2021-06-21 02:24:57.144650+00:00   1048276   
3552     60553 2021-05-12 13:15:43.421237+00:00   1015843   
3838     52363 2021-06-21 01:00:15.009526+00:00   1047960   
3839    136665 2021-06-21 01:00:20.717898+00:00   1047963   
3840     45445 2021-06-21 01:01:24.606873+00:00   1047985   
3841     51044 2021-06-21 01:02:09.290831+00:00   1047996   
3842    136935 2021-06-21 01:03:30.135052+00:00   1048008   
3843     48879 2021-06-21 01:05:08.565879+00:00   1048016   
3844    138451 2021-06-21 01:13:14.523692+00:00   1048058   
3848    137510 2021-06-21 01:13:22.343163+00:00   1048060   
3849    121556 2021-06-21 01:15:03.532632+00:00   1048073   
3850     24073 2021-06-21 01:19:47.115436+00:00   1048094   
3851    139104 2021-06-21 01:21:22.656785+00:00   1048110   
3852     80714 2021-06-21 01:23:35.889704+00:00   1048119   
3853     87097 2021-06-21 01:23:56.480943+00:00   1048120   
3855    137790 2021-06-21 01:19:41.738448+00:00   1048092   
3856     91769 2021-06-21 01:25:27.640027+00:00   1048128   
3857    124503 2021-06-21 01:25:29.317054+00:00   1048129   
3858    139037 2021-06-21 01:39:06.938698+00:00   1048158   
3859    104273 2021-06-21 01:39:51.085044+00:00   1048160   
3860     77874 2021-06-21 01:40:20.141895+00:00   1048164   
3862    138981 2021-06-21 01:42:00.905369+00:00   1048173   
3863     74457 2021-06-21 01:43:11.193430+00:00   1048177   
3864     96481 2021-06-21 01:46:24.010241+00:00   1048181   
3868     76285 2021-06-21 01:50:33.210642+00:00   1048191   
3870     48659 2021-06-21 01:49:53.700930+00:00   1048187   
3871     94292 2021-06-21 02:05:11.522082+00:00   1048219   
3873     47924 2021-06-21 02:11:52.887910+00:00   1048236   
3874    133219 2021-06-21 02:18:58.354178+00:00   1048252   
3875    132324 2021-06-21 02:19:08.127578+00:00   1048253   
3876    110237 2021-06-21 02:29:36.996661+00:00   1048287   
3877    130257 2021-06-21 02:38:06.255460+00:00   1048308   
3878    138578 2021-06-21 02:24:00.718124+00:00   1048271   
3880     67950 2021-06-21 02:39:04.583298+00:00   1048313   
3882     87789 2021-06-21 02:39:46.430145+00:00   1048315   
3883     24772 2021-06-21 02:45:29.111626+00:00   1048330   
3884    133791 2021-06-21 02:48:11.506768+00:00   1048337   
3885    114523 2021-06-21 02:49:42.369717+00:00   1048343   
3886     36592 2021-06-21 02:52:05.741381+00:00   1048349   
3888     87097 2021-06-21 02:53:05.624687+00:00   1048354   
3890     86758 2021-06-21 02:55:19.073716+00:00   1048357   
3891    136636 2021-06-21 02:57:33.534400+00:00   1048367   
3892    135643 2021-06-21 02:58:18.377794+00:00   1048370   
3893    138860 2021-06-21 02:58:43.743821+00:00   1048373   
3894    131449 2021-06-21 02:59:07.083115+00:00   1048374   
3895     24248 2021-06-21 02:59:50.640772+00:00   1048380   
4126    131334 2021-06-21 01:09:08.816201+00:00   1048037   
4138     73697 2021-06-21 02:43:28.848373+00:00   1048324   
4148     76416 2021-06-21 02:59:32.285992+00:00   1048378   
4232    137792 2021-06-21 01:14:26.702305+00:00   1048067   
4300     65202 2021-06-21 01:42:39.679143+00:00   1048175   
4455    105273 2021-05-12 01:19:24.777416+

- L3 ADMIN 기준으로 확인 결과, 참고값이 0인 경우 파생값이 맞았고, 0이 아닌데 파생값이 0인 경우도 파생값이 맞았다. 
- 다만, 이 사례에서도 여전히 15409 사례는 참고값쪽이 맞아 확인이 필요하다. 15409 사례가 뭔가 특이한것일까?

In [40]:
test = pd.merge(dataframe_fee[['order_id',
                               'quality_inspection_fee_amount',
                               'quality_inspection_fee_supply',
                              'quality_inspection_fee_vat']],
                        order_prod_fee.groupby('order_id')[cols].sum().reset_index(),
                        on = 'order_id',
                        suffixes=['', '_unit'])

In [41]:
test.loc[((test['quality_inspection_fee_amount'] - test['quality_inspection_fee_amount_unit']) != 0) & (
    ~(test['quality_inspection_fee_amount'] - test['quality_inspection_fee_amount_unit']).isna()), [
                                                                                'order_id',
                                                                                'quality_inspection_fee_amount',
                                                                                'quality_inspection_fee_amount_unit']]

order_id  quality_inspection_fee_amount  \
0       1076769                        2200.00   
1       1076770                         500.00   
2       1076771                         500.00   
3       1076772                        1500.00   
4       1076773                        2400.00   
...         ...                            ...   
69120   1076758                        4800.00   
69121   1076760                        2200.00   
69122   1076761                        1900.00   
69123   1076766                        5700.00   
69125   1076768                         500.00   

       quality_inspection_fee_amount_unit  
0                                    0.00  
1                                    0.00  
2                                    0.00  
3                                    0.00  
4                                    0.00  
...                                   ...  
69120                                0.00  
69121                                0.00  
69122                                0.00  
69123                             5000.00  
69125                                0.00  

[60401 rows x 3 columns]

- inspection을 기준으로 파생값을 생산할경우, 오차율이 상당하다.

In [42]:
# 'status' = -1인 경우 제외

dataframe_fee_reduced = pd.merge(dataframe_order.rename({'id' : 'order_id'}, axis = 1)['order_id'],
                                     dataframe_fee[['order_id',
                                                    'quality_inspection_fee_amount',
                                                    'quality_inspection_fee_supply',
                                                    'quality_inspection_fee_vat']],
                                    on = 'order_id')

In [43]:
order_prod_fee['quality_inspection_fee_amount'] = order_prod_fee['inspection'] * (
                                                  order_prod_fee['unit_quality_inspection_fee_amount'])

In [44]:
test = pd.merge(dataframe_fee_reduced[['order_id',
                               'quality_inspection_fee_amount',
                               'quality_inspection_fee_supply',
                              'quality_inspection_fee_vat']],
                        order_prod_fee.groupby('order_id')[cols].sum().reset_index(),
                        on = 'order_id',
                        suffixes=['', '_unit'])

In [45]:
test.loc[((test['quality_inspection_fee_amount'] - test['quality_inspection_fee_amount_unit']) != 0) & (
    ~(test['quality_inspection_fee_amount'] - test['quality_inspection_fee_amount_unit']).isna()), [
                                                                                'order_id',
                                                                                'quality_inspection_fee_amount',
                                                                                'quality_inspection_fee_amount_unit']].iloc[3000 : 3100]

order_id  quality_inspection_fee_amount  \
6249   1053907                        3000.00   
6251   1072391                        3400.00   
6253   1027258                       48150.00   
6257   1054515                        1000.00   
6258   1053910                        8000.00   
6262   1027722                        4000.00   
6268   1027954                        2200.00   
6270   1050570                        2800.00   
6272   1050572                        1000.00   
6274   1021299                        7800.00   
6275   1050590                        1000.00   
6277   1050588                        9000.00   
6278   1072388                        1000.00   
6279   1050574                        6100.00   
6281   1021313                         500.00   
6282   1021322                        1500.00   
6283   1050585                        4300.00   
6285   1029336                        8400.00   
6286   1050579                        3500.00   
6287   1066147                       13300.00   
6289   1021371                        7100.00   
6295   1021400                        5500.00   
6297   1054517                       29600.00   
6298   1069594                        3500.00   
6299   1050608                        5000.00   
6302   1021504                       19200.00   
6304   1030313                       37200.00   
6310   1021548                        4000.00   
6315   1021556                        4600.00   
6317   1021600                       97500.00   
6318   1021592                        3000.00   
6325   1021603                        4100.00   
6326   1050636                        2000.00   
6330   1036635                        2600.00   
6341   1053936                        4900.00   
6342   1021708                        3200.00   
6343   1021672                        2700.00   
6346   1030874                        3400.00   
6347   1021702                        5100.00   
6351   1021673                       12600.00   
6352   1031651                       11300.00   
6353   1050651                        1000.00   
6354   1067715                        5500.00   
6358   1032196                       15700.00   
6359   1048664                       59570.00   
6364   1036255                        6100.00   
6365   1013306                       42700.00   
6369   1042032                       25500.00   
6370   1049328                       24185.00   
6373   1014750                        4200.00   
6375   1031277                        1500.00   
6378   1031463                        8100.00   
6379   1068265                        6000.00   
6380   1031306                       23900.00   
6381   1028125                        4000.00   
6388   1060749                        4100.00   
6389   1033566                       10000.00   
6390   1030220                       54200.00   
6391   1032227                       10000.00   
6392   1060754                      220600.00   
6394   1032243                         500.00   
6395   1032230                        2000.00   
6396   1060756                        1000.00   
6397   1038887                        3950.00   
6399   1050683                        1900.00   
6401   1032237                         500.00   
6402   1045596                        3400.00   
6403   1069226                        7400.00   
6404   1032244                        7000.00   
6405   1038961                       10900.00   
6411   1032250                        7000.00   
6412   1047239                        1000.00   
6413   1040412                        3100.00   
6416   1069413                        4400.00   
6418   1036406                        1500.00   
6421   1032257                        5500.00   
6423   1032261                        7700.00   
6424   1036398                       15700.00   
6426   1031637                       15900.00   
6427   1032273                       18100.00   
6431   1036386                       

- 일부를 샘플링(n = 100)하여 확인해본 결과, L3에선 구한 파생값 기준으로 표출이 되고 있음을 확인하였다.

#### 결론 1 : 가설을 다소 수정할경우, 아래의 관계는 대부분 성립한다.

1) orders의 finalHandlingFee = tb_order_prod_fee의 unit_handling_Fee * order_product의 finalQuantity

2) orders의 finalSubtotalKRW = tb_order_prod_fee의 unit_product_supply * order_product의 finalQuantity
**(단, unit_final_product_supply 값이 존재할경우 해당 값 우선)**

3) orders의 finalTax = tb_order_prod_fee의 unit_product_vat * order_product의 finalQuantity
**(단, unit_final_product_vat 값이 존재할경우 해당 값 우선)**

4) order_fee의 quality_inspection_fee_amount = tb_order_prod_fee의 unit_quality_inspection_fee_amount * order_product의 finalQuantity

### - 가설2 : 아래의 관계는 성립할것이다.

1) orders의 HandlingFeeKRW = tb_order_prod_fee의 unit_handling_Fee_amount * order_product의 quantity

2) orders의 SubtotalKRW = tb_order_prod_fee의 unit_product_amount * order_product의 quantity

3) orders의 taxKRW = tb_order_prod_fee의 unit_product_vat * order_product의 Quantity

4) order_fee의 quality_inspection_fee = tb_order_prod_fee의 unit_quality_inspection_fee * order_product의 Quantity 

In [46]:
cols = list(set(dataframe_prod_fee.columns).difference(
    set(['id','created_at','creator_id','updated_at',' updater_id', 'use_yn','order_id', 'item_type','item_id'])))

cols.sort()

In [47]:
dataframe_prod_fee[cols] = dataframe_prod_fee[cols].astype('float')

dataframe_prod_fee = dataframe_prod_fee.rename({'item_id' : 'orderProductId'}, axis = 1)

In [48]:
order_prod_fee = pd.merge(dataframe_order_prod, 
                        dataframe_prod_fee,
                         on = ['orderProductId'])

In [49]:
order_prod_fee['quality_inspection_fee_amount'] = order_prod_fee['unit_quality_inspection_fee_amount'] * (
    order_prod_fee['cusumOrderQuantity'])
order_prod_fee['handlingFeeKRW'] = order_prod_fee['unit_handling_fee_amount'] * order_prod_fee['cusumOrderQuantity']
order_prod_fee['subtotalKRW'] = order_prod_fee['unit_product_supply'] * order_prod_fee['cusumOrderQuantity']
order_prod_fee['taxKRW'] = order_prod_fee['unit_product_vat'] * order_prod_fee['cusumOrderQuantity']

In [50]:
test = pd.merge(order_prod_fee.groupby('orderId')[[ 'quality_inspection_fee_amount',
                                                    'handlingFeeKRW',
                                                    'subtotalKRW',
                                                    'taxKRW']].sum().reset_index(),
         dataframe_order.rename({'id' : 'orderId'}, axis = 1),
         on = 'orderId',
         suffixes = ('', '_unit'))

In [51]:
test['quality_inspection_fee_amount'] = test['quality_inspection_fee_amount'].astype('float')
test['handlingFeeKRW_unit'] = test['handlingFeeKRW_unit'].astype('float')
test['subtotalKRW_unit'] = test['subtotalKRW_unit'].astype('float')
test['taxKRW_unit'] = test['taxKRW_unit'].astype('float')

In [52]:
test = test[np.sort(test.columns)]

In [53]:
print((test['handlingFeeKRW'] - test['handlingFeeKRW_unit']).unique())
print((test['subtotalKRW'] - test['subtotalKRW_unit']).unique())
print((test['taxKRW'] - test['taxKRW_unit']).unique())

[    0. -2080.]
[     0. -26000.]
[0.]


In [54]:
test.loc[(test['handlingFeeKRW'] - test['handlingFeeKRW_unit']) != 0,  ['orderId',
                                                                        'handlingFeeKRW',
                                                                        'handlingFeeKRW_unit']]

orderId  handlingFeeKRW  handlingFeeKRW_unit
3448  1022136       782720.00            784800.00

위 사례의 경우, L3에선 < _unit > 기준으로 표시되고 있다.

In [55]:
test.loc[(test['subtotalKRW'] - test['subtotalKRW_unit']) != 0, ['orderId',
                                                                        'subtotalKRW',
                                                                        'subtotalKRW_unit']]

orderId  subtotalKRW  subtotalKRW_unit
3448  1022136   9784000.00        9810000.00

위 사례 중

1) 첫 번째 사례(orderId = 1022136)은 위에서 살펴본대로 < _unit >을 따른다.

In [56]:
test.loc[(test['taxKRW'] - test['taxKRW_unit']) != 0, ['orderId',
                                                        'taxKRW',
                                                        'taxKRW_unit']]

Empty DataFrame
Columns: [orderId, taxKRW, taxKRW_unit]
Index: []

In [57]:
test = pd.merge(dataframe_fee_reduced[['order_id',
                               'quality_inspection_fee_amount',
                               'quality_inspection_fee_supply',
                              'quality_inspection_fee_vat']],
                        order_prod_fee.groupby('order_id')[cols].sum().reset_index(),
                        on = 'order_id',
                        suffixes=['', '_unit'])

In [58]:
test.loc[((test['quality_inspection_fee_amount'] - test['quality_inspection_fee_amount_unit']) != 0) & (
    ~(test['quality_inspection_fee_amount'] - test['quality_inspection_fee_amount_unit']).isna()), [
                                                                                'order_id',
                                                                                'quality_inspection_fee_amount',
                                                                                'quality_inspection_fee_amount_unit']].iloc[3000 : 3100]

Empty DataFrame
Columns: [order_id, quality_inspection_fee_amount, quality_inspection_fee_amount_unit]
Index: []

#### 결론 2 : 아래의 관계는 대부분 성립한다.

1) orders의 HandlingFeeKRW = tb_order_prod_fee의 unit_handling_Fee_amount * order_product의 quantity

2) orders의 SubtotalKRW = tb_order_prod_fee의 unit_product_amount * order_product의 quantity

3) orders의 taxKRW = tb_order_prod_fee의 unit_product_vat * order_product의 quantity

4) order_fee의 quality_inspection_fee = tb_order_prod_fee의 unit_quality_inspection_fee * order_product의 Quantity

### 가설 3 : 아래의 관계는 대부분 성립할것이다

1) order_fee의 dc_handling_fee_amount = order_product_fee의 dc_unit_handling_fee * order_product의 finalQuantity

2) order_fee의 dc_quality_inspection_fee_amount = order_product_fee의 dc_unit_quality_inspection_fee * order_product의 finalQuantity

3) order_fee의 dc_product_amount = order_product_fee의 dc_unit_product_supply * order_product의 finalQuantity

In [59]:
order_prod_fee['dc_handling_fee_amount'] = order_prod_fee['dc_unit_handling_fee_amount'] * (
                                            order_prod_fee['cusumPurchaseQuantity'])
order_prod_fee['dc_quality_inspection_fee_amount'] = order_prod_fee['dc_unit_quality_inspection_fee_amount'] * (
                                                    order_prod_fee['cusumPurchaseQuantity'])
order_prod_fee['dc_product_amount'] = order_prod_fee['dc_unit_product_amount'] * order_prod_fee['cusumPurchaseQuantity']

In [60]:
test = pd.merge(order_prod_fee.groupby('orderId')[['dc_handling_fee_amount',
                                                    'dc_quality_inspection_fee_amount',
                                                    'dc_product_amount']].sum().reset_index(),
         dataframe_fee.rename({'order_id' : 'orderId'}, axis = 1),
         on = 'orderId',
         suffixes = ('_unit', ''))

In [61]:
test = test[np.sort(test.columns)]

In [62]:
(test['dc_product_amount_unit'] - test['dc_product_amount']).unique()

array([     0., -68685.,  -9240.,  -6300.,  -1320.])

In [63]:
test.loc[(test['dc_product_amount_unit'] - test['dc_product_amount']) != 0, ['orderId',
                                                                             'dc_product_amount_unit',
                                                                            'dc_product_amount']]

orderId  dc_product_amount_unit  dc_product_amount
40236  1049893                    0.00           68685.00
40237  1049894                    0.00            9240.00
40239  1049896                    0.00            9240.00
40241  1049898                    0.00            9240.00
40249  1049906                    0.00            6300.00
40257  1049918                    0.00            1320.00

In [64]:
(test['dc_handling_fee_amount_unit'] - test['dc_handling_fee_amount']).unique()

array([     0., -11360.,  -4160., ...,   -226.,  -1487.,   -196.])

In [65]:
test.loc[(test['dc_handling_fee_amount_unit'] - test['dc_handling_fee_amount']) != 0, ['orderId',
                                                                             'dc_handling_fee_amount_unit',
                                                                            'dc_handling_fee_amount']]

orderId  dc_handling_fee_amount_unit  dc_handling_fee_amount
5249   1013312                         0.00                11360.00
5251   1013316                      7200.00                11360.00
5267   1013332                         0.00                 9280.00
5637   1013730                         0.00                22320.00
5640   1013733                         0.00                22320.00
...        ...                          ...                     ...
68321  1079013                        74.00                  270.00
68512  1079208                         0.00                   50.00
68670  1079380                         0.00                 1326.00
68732  1079444                         0.00                   38.00
68768  1079480                         0.00                  180.00

[4457 rows x 3 columns]

### 결론 3 : 최종구매수량과 곱하는 방법은 타당하지 않은것으로 보인다.

### 수정가설 3 : 아래의 관계는 대부분 성립할것이다

1) order_fee의 dc_handling_fee_amount = order_product_fee의 dc_unit_handling_fee * order_product의 Quantity

2) order_fee의 dc_quality_inspection_fee_amount = order_product_fee의 dc_unit_quality_inspection_fee * order_product의 Quantity

3) order_fee의 dc_product_amount = order_product_fee의 dc_unit_product_supply * order_product의 Quantity


In [66]:
order_prod_fee['dc_handling_fee_amount'] = order_prod_fee['dc_unit_handling_fee_amount'] * (
                                            order_prod_fee['cusumOrderQuantity'])
order_prod_fee['dc_quality_inspection_fee_amount'] = order_prod_fee['dc_unit_quality_inspection_fee_amount'] * (
                                                    order_prod_fee['cusumOrderQuantity'])
order_prod_fee['dc_product_amount'] = order_prod_fee['dc_unit_product_amount'] * order_prod_fee['cusumOrderQuantity']


In [67]:
dataframe_fee_reduced = pd.merge(dataframe_order.rename({'id' : 'order_id'}, axis = 1)['order_id'],
                         dataframe_fee,
                         on = 'order_id')

In [68]:
test = pd.merge(order_prod_fee.groupby('orderId')[['dc_handling_fee_amount',
                                                    'dc_quality_inspection_fee_amount',
                                                    'dc_product_amount']].sum().reset_index(),
                 dataframe_fee_reduced.rename({'order_id' : 'orderId'}, axis = 1),
                 on = 'orderId',
                 suffixes = ('_unit', ''))

In [69]:
test = test[np.sort(test.columns)]

In [70]:
test.loc[((test['dc_handling_fee_amount'] - test['dc_handling_fee_amount_unit']) == 0) & (
        test['dc_handling_fee_amount'] > 0), ['orderId',
                                                        'dc_handling_fee_amount',
                                                        'dc_handling_fee_amount_unit']]

orderId  dc_handling_fee_amount  dc_handling_fee_amount_unit
1318   1013316                11360.00                     11360.00
1404   1013751                22320.00                     22320.00
1510   1014111                25200.00                     25200.00
2656   1018980                16010.00                     16010.00
2657   1018991                 1600.00                      1600.00
...        ...                     ...                          ...
16639  1077891                  127.00                       127.00
16865  1078798                  635.00                       635.00
16935  1079013                  270.00                       270.00
16960  1079129                  519.00                       519.00
16976  1079210                   50.00                        50.00

[1399 rows x 3 columns]

In [71]:
test.loc[(test['dc_handling_fee_amount_unit'] - test['dc_handling_fee_amount']) != 0, ['orderId',
                                                                                    'dc_handling_fee_amount',
                                                                                    'dc_handling_fee_amount_unit']]

orderId  dc_handling_fee_amount  dc_handling_fee_amount_unit
2691   1019138                    0.00                      9490.00
2873   1019913                    0.00                      4460.00
2890   1019968                    0.00                      2060.00
9665   1048824                    0.00                      7400.00
9718   1049019                    0.00                      1080.00
9763   1049136                    0.00                      3650.00
12842  1062029                    0.00                      2934.00
13076  1062844                    0.00                     10841.00
14875  1070396                    0.00                      4640.00

In [72]:
test.loc[((test['dc_quality_inspection_fee_amount'] - test['dc_quality_inspection_fee_amount_unit']) == 0) & (
        test['dc_quality_inspection_fee_amount'] > 0), ['orderId',
                                                        'dc_quality_inspection_fee_amount',
                                                        'dc_quality_inspection_fee_amount_unit']]

orderId  dc_quality_inspection_fee_amount  \
112    1008383                          12930.00   
158    1008561                           2955.00   
203    1008691                           8700.00   
204    1008692                          31260.00   
205    1008694                           7860.00   
...        ...                               ...   
16865  1078798                           1496.00   
16935  1079013                            340.00   
16960  1079129                            969.00   
16976  1079210                            170.00   
17102  1079836                           4305.00   

       dc_quality_inspection_fee_amount_unit  
112                                 12930.00  
158                                  2955.00  
203                                  8700.00  
204                                 31260.00  
205                                  7860.00  
...                                      ...  
16865                                1496.00  
16935                                 340.00  
16960                                 969.00  
16976                                 170.00  
17102                                4305.00  

[765 rows x 3 columns]

In [73]:
test.loc[((test['dc_quality_inspection_fee_amount'] - test['dc_quality_inspection_fee_amount_unit']) != 0), ['orderId',
                                                                                    'dc_quality_inspection_fee_amount',
                                                                                    'dc_quality_inspection_fee_amount_unit']]

orderId  dc_quality_inspection_fee_amount  \
3448   1022136                         392400.00   
12842  1062029                              0.00   
13076  1062844                              0.00   
14875  1070396                              0.00   

       dc_quality_inspection_fee_amount_unit  
3448                               391360.00  
12842                                 770.00  
13076                                7700.00  
14875                                 483.00

In [74]:
test.loc[test['dc_product_amount'] > 0,  ['orderId',
                                    'dc_product_amount',
                                    'dc_product_amount_unit']]

Empty DataFrame
Columns: [orderId, dc_product_amount, dc_product_amount_unit]
Index: []

### 결론 4 : 아래의 관계는 대부분 성립한다.

1) order_fee의 dc_handling_fee_amount = order_product_fee의 dc_unit_handling_fee * order_product의 Quantity

(오차사례 / 0 이상 전체 사례 : 9 / 1399)

2) order_fee의 dc_quality_inspection_fee_amount = order_product_fee의 dc_unit_quality_inspection_fee * order_product의 Quantity

(오차 사례 / 0 이상 전체 사례 : 4 / 765)

- 값이 0 이상인데 오차인 경우는 대부분 참고값이 0인 경우인데 파생값은 0 이상인 경우였고, 참고값이 0이 아닌데도 오차가 존재하는 경우는 한건 존재하나, 오차 수준은 매우 작다.

- 다만, 매우 작으나 오차가 난 이유에 대해서는 별도 확인이 필요하다. -> L3상에선 392,400원 할인으로, 참고값 기준으로 값이 표시되고 있다. 

### 결론 5: 아래의 관계는 확인이 불가능하다

1) order_fee의 dc_product_amount = order_product_fee의 dc_unit_product_supply * order_product의 Quantity

- dc_product_amount가 0 이상인 값이 존재하지 않아서, 값이 타당한지 아닌지 확인이 어렵다.

# 3. 결론

### 1) final 관련

#### (1) final관련 변수들의 경우, 아래의 관계는 일반적으로 성립한다.

- orders의 finalHandlingFee = tb_order_prod_fee의 unit_handling_Fee * order_product의 finalQuantity

- orders의 finalSubtotalKRW = tb_order_prod_fee의 unit_product_supply * order_product의 finalQuantity **(단, unit_final_product_supply 값이 존재할경우 해당 값 우선)**

- orders의 finalTax = tb_order_prod_fee의 unit_product_vat * order_product의 finalQuantity **(단, unit_final_product_vat 값이 존재할경우 해당 값 우선)**

- order_fee의 quality_inspection_fee_amount = tb_order_prod_fee의 unit_quality_inspection_fee_amount * order_product의 finalQuantity

### 2) 최초 생성값 관련

#### (1) 최초 생성값과 관련된 변수들의 경우, 아래의 관계는 일반적으로 성립한다.

- orders의 HandlingFeeKRW = tb_order_prod_fee의 unit_handling_Fee_amount * order_product의 quantity

- orders의 SubtotalKRW = tb_order_prod_fee의 unit_product_amount * order_product의 quantity

- orders의 taxKRW = tb_order_prod_fee의 unit_product_vat * order_product의 quantity

- order_fee의 quality_inspection_fee = tb_order_prod_fee의 unit_quality_inspection_fee * order_product의 Quantity

### 3) 할인 관련값 관련

#### (1) 할인과 관련된 변수들의 경우, 아래의 관계는 일반적으로 성립한다.

- order_fee의 dc_handling_fee_amount = order_product_fee의 dc_unit_handling_fee * order_product의 finalQuantity

- order_fee의 dc_quality_inspection_fee_amount = order_product_fee의 dc_unit_quality_inspection_fee * order_product의 finalQuantity

- order_fee의 dc_product_amount = order_product_fee의 dc_unit_product_supply * order_product의 finalQuantity